In [ ]:
! pip install bs4
! pip install fuzzywuzzy
! pip install geohash
! pip install python-geohash
! pip install pygeohash
! pip install python-Levenshtein
! pip install proximityhash
import urllib.request
import requests
import numpy as np
from urllib.request import Request, urlopen
import pandas as pd
import re
from bs4 import BeautifulSoup
from random import randint
import time
import math
import csv
import string
# import json
# from selenium import webdriver
# from selenium.webdriver.support.ui import Select 
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# import Geohash
# import pygeohash as pgh
# import proximityhash

In [ ]:
#Parsing webpage
url_front = "https://giantonline.com.sg/"
req = Request(url_front, headers={'User-Agent': 'Chrome//5.0'})
page = urlopen(req)
soup = BeautifulSoup(page)
all_left_menu_item = soup.find('div', {'class' : 'left-menu'})


#Converting cateogries into string for regex & to generate the broad categories
broad_categories = all_left_menu_item.findAll('a', {'class' : 'department'})
for i in range(0,len(broad_categories)):
    broad_categories[i] = str(broad_categories[i])
    broad_categories[i] = re.findall(r'">(.*?)</a>', broad_categories[i])[0]
    broad_categories[i] = broad_categories[i].replace(", ", "-")
    broad_categories[i] = broad_categories[i].replace(" &amp; ", "-")
    if broad_categories[i][-1] == " ":
        broad_categories[i] = broad_categories[i][0:-1]
    broad_categories[i] = broad_categories[i].replace(" ", "-")

#Cos giant fked-up, name their product differently
broad_categories[1] = 'Beers-Wines-Spirits'

#Converting broad categories into smaller categories. For e.g. 'Rice' is under 'Food Pantry'    
narrow_categories = all_left_menu_item.findAll('li', {'class' : 'dropdown category'})
narrow_categories = narrow_categories[1:]
for i in range(0,len(narrow_categories)):
    narrow_categories[i] = str(narrow_categories[i])
    narrow_categories[i] = re.findall(r'">(.*?)</a>', narrow_categories[i])[1:]
    for x in range(0,len(narrow_categories[i])):
        narrow_categories[i][x] = narrow_categories[i][x].replace(", ", "-")
        narrow_categories[i][x] = narrow_categories[i][x].replace(" &amp; ", "-")
        narrow_categories[i][x] = narrow_categories[i][x].replace(" ", "-")
        narrow_categories[i][x] = narrow_categories[i][x].replace(".", "")
        narrow_categories[i][x] = narrow_categories[i][x].replace("'", "")

#Again cos giant fked-up, name their products differently
narrow_categories[0][-1] = 'international-groceries'
narrow_categories[2][-1] = 'chocolate-malts-adult-milk'

# Combining the list of URLs to form the webpage for only the first page        
combine_categories = list()
for i in range(0,len(broad_categories)):
    for x in range(0, len(narrow_categories[i])):
        word = str('https://giantonline.com.sg/' + str(broad_categories[i]) + '/' + str(narrow_categories[i][x]) + '/')
        combine_categories.append(word)

# Coming up with the full list of product with page
combine_cat_page = list()
for x in range(0,len(combine_categories)):
    for i in range(0,30):
        webpagelink = combine_categories[x] + '?Product_page=' + str(i)
        combine_cat_page.append(webpagelink)



print(len(combine_cat_page))
# print(all_left_menu_item)
# len(narrow_categories)
# print(narrow_categories[1])
# broad_categories
print(combine_cat_page)

In [ ]:
#Conducting actual webscraping
product_details = {}
df = pd.DataFrame()
for i in range(0,len(combine_cat_page)):
    req = Request(combine_cat_page[i], headers={'User-Agent': 'Chrome//5.0'})
    page = urlopen(req)
    soup = BeautifulSoup(page)
    items = soup.findAll('li', {'class' : 'col-xs-6 col-sm-4 col-md-3 col-lg-2 open-product-detail algolia-click'})
    current_page = items[0].find('a', {'class' : "product-link"}).get_text()
    if i == 0 or current_page != previous_page:
        for x in range(0,len(items)):
            try:
                #image source
                src = str(items[x].find('img'))
                product_details['Image'] = re.findall(r'src="(.*?)"', src)[0]
            except:
                src = "No Image Link"
                product_details['Image'] = src
            try:
                #brand name
                brand = items[x].find('a', {'class' : 'to-brand-page'}).get_text()
                product_details['Brand'] = brand
            except:
                brand = 'Brandless'
                product_details['Brand'] = brand
            try:
                #further product info
                details = items[x].find('a', {'class' : "product-link"}).get_text()
                product_details['Further Details'] = details
            except:
                details = 'No Further Details'
                product_details['Further Details'] = details
            try:
                #weight of product
                size = items[x].find('span', {'class' : 'size'}).get_text()
                product_details['Size'] = size
            except:
                size = 'No Size'
                product_details['Size'] = size
            try:
                #any discounts
                discount = items[x].find('span', {'class' : 'product-discount-label'}).get_text()
                product_details['Discount'] = discount
            except:
                discount = 'No Discount'
                product_details['Discount'] = discount
            try:
                #Happens when there's a discount
                items_x = str(items[x])
                new_price = re.findall(r'<div class="price product-price red" data-price="(.*?)">', items_x)[0]
                old_price = re.findall(r'<span class="price old-price">(.*?)</span>', items_x)[0]
                product_details['Current Price'] = new_price
                product_details['Old Price'] = old_price
            except:
                try:
                    new_price = items[x].find('div', {'class' : 'price product-price'}).get_text()
                    product_details['Current Price'] = new_price
                    product_details['Old Price'] = 'No Discount'
                except:
                    new_price = items[x].find('div', {'class' : 'content_price'}).get_text().rstrip().lstrip()
                    product_details['Current Price'] = new_price
                    product_details['Old Price'] = 'No Discount'
            try:
                #link to goods under the brand
                brand_goods = str(items[x].find('a', {'class' : 'to-brand-page'}))
                brand_goods = re.findall(r'href="(.*?)">', brand_goods)[0]
                product_details['Brands Goods'] = brand_goods
            except:
                product_details['Brands Goods'] = 'No Page for Brand Goods'
            try:
                # Links to more details
                more_details = str(items[x])
                more_details = re.findall(r'<a class="product-link" href="(.*?)">', more_details)[0]
                more_details = 'https://giantonline.com.sg' + more_details
                product_details['Link to More Details'] = more_details
            except:
                product_details['Link to More Details'] = 'No Link'
            try:
                outstock = items[13].find('div', {'class' : 'btn add-cart btn-default out-of-stock'}).get_text().rstrip().lstrip()
                product_details['Stock'] = outstock
            except:
                product_details['Stock'] = "In Stock"
            try:
                comb_info = combine_cat_page[0].split('_page=')
                page_number = comb_info[0]
                comb_cat = re.findall(r'https://giantonline.com.sg/(.*?)/?Product', comb_info[0])
                broad_cat = comb_cat[0].split('/')[0]
                narrow_cat = comb_cat[0].split('/')[1]
                product_details['Page Number'] = page_number
                product_details['Broad Category'] = broad_cat
                product_details['Narrow Category'] = narrow_cat
            except:
                product_details['Page Number'] = 'No Page'
                product_details['Broad Category'] = 'No Broad Cat'
                product_details['Narrow Category'] = 'No Narrow Cat'
            df = df.append(product_details, ignore_index = True)
        previous_page = current_page     
    print(i)


In [ ]:
print(df)
df.to_csv('Giant Supermarket Scrape.csv', sep=',')